In [32]:
from datetime import datetime, timezone
from typing import List
from dataclasses import dataclass
from os.path import join
import numpy as np
import pandas as pd
from pynwb import NWBFile, NWBHDF5IO, TimeSeries
from pynwb.file import Subject
from pynwb.ecephys import ElectricalSeries, ElectrodeGroup
from pynwb.behavior import BehavioralEvents
import nixio

In [33]:
@dataclass(frozen=True)
class SessionContext:
    subject: int
    session: int
    nix: nixio.File
    nwb: NWBFile

In [34]:
def main() -> SessionContext:
    ctx = create_context(1, 1)
    write_subject(ctx)
    write_eeg_electrodes(ctx)
    write_eeg_measurements(ctx)
    write_ieeg_electrodes(ctx)
    write_ieeg_measurements(ctx)
    write_trial_data(ctx)
    write_behavioral_events(ctx)
    # TODO: Add EEG and iEEG events?
    # TODO: Add waveforms
    # write_nwb(ctx)
    return ctx

In [35]:
def create_context(subject: int, session: int) -> SessionContext:
    nix = _read_nix(subject, session)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description=_get_task(nix),
        identifier=f"Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject{subject:02}_session{session:02}",
        session_start_time=datetime(2019, 3, 27, 12, 00, tzinfo=timezone.utc),  # TODO: Get this from Johannes
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        related_publications=_get_related_publications(nix)
    )
    return SessionContext(subject, session, nix, nwb)


In [36]:
def _read_nix(subject: int, session: int) -> nixio.File:
    _nix_dir = join("/", "mnt", "c", "Users", "conta", "git", "USZ_NCH", "Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
                    "data_nix")
    _file_path = join(_nix_dir, f"Data_Subject_{subject:02}_Session_{session:02}.h5")
    return nixio.File.open(_file_path, nixio.FileMode.ReadOnly)

In [37]:
def _get_task(nix: nixio.File) -> str:
    task = nix.sections["Task"].props
    task_name = task["Task name"].values[0]
    # Broken UTF-8 in file
    task_desc = "The task is a modified Sternberg task in which the encoding of memory items, maintenance, and recall were temporally separated. Each trial starts with a fixation period ([-6, -5] s), followed by the stimulus ([-5, -3] s). The stimulus consists of a set of eight consonants at the center of the screen. The middle four, six, or eight letters are the memory items,which determine the set size for the trial (4, 6, or 8, respectively). The outer positions are filled with “X,” which is never a memory item. After the stimulus, the letters disappear from the screen, and the maintenance interval starts ([-3, 0] s).A fixation square is shown throughout fixation, encoding, and maintenance. After maintenance, a probe letter is presented. The subjects respond with a button press to indicate whether the probe was part of the stimulus.The subjects are instructed to respond as rapidly as possible without making errors. The hand used for the response is counterbalanced across subjects within the clinical constraints. After the response, the probe is turned off, and the subjects receive acoustic feedback regarding whether their response was correct or incorrect. Before initiating the next trial, the subjects were encouraged to blink and relax. The subjects perform 50 trials in one session, which last approximately 10 min. Trials with different set sizes are presented in a random order,with the single exception that a trial with an incorrect response is always followed by a trial with a set size of 4."
    task_url = task["Task URL"].values[0]
    return f"Task Name: {task_name}\nTask Description: {task_desc}\nTask URL: {task_url}"

In [38]:
def _get_related_publications(nix: nixio.File) -> List[str]:
    related_publications = nix.sections["General"].sections["Related publications"].props
    names_and_dois = zip(related_publications["Publication name"].values,
                         related_publications["Publication DOI"].values)
    return [f"{name.strip()} ({doi.strip()})" for (name, doi) in names_and_dois]

In [39]:
def write_subject(ctx: SessionContext):
    subject = ctx.nix.sections["Subject"].props
    age = subject["Age"].values[0]
    sex = subject["Sex"].values[0]
    ctx.nwb.subject = Subject(
        subject_id=f"{ctx.subject:02}",
        age=f"P{int(age)}Y",
        description=_get_subject_description(ctx),
        species="Homo sapiens",
        sex=_get_sex(sex),
    )

In [40]:
def _get_sex(raw: str) -> str:
    male = ["male", "m", "männlich", "maennlich", "mannlich", "männchen", "mannchen"]
    female = ["female", "f", "weiblich", "weibchen"]
    sex = raw.lower()
    return "M" if sex in male else "F" if sex in female else "O"

In [41]:
def _get_subject_description(ctx: SessionContext) -> str:
    subject = ctx.nix.sections["Subject"].props
    handedness = subject["Handedness"].values[0]
    pathology = subject["Pathology"].values[0]
    depth_electrodes = subject["Depth electrodes"].values[0]
    electrodes_in_soz = subject["Electrodes in seizure onset zone (SOZ)"].values[0]
    return f"Handedness: {handedness}\nPathology: {pathology}\nDepth electrodes: {depth_electrodes}\nElectrodes in seizure onset zone (SOZ): {electrodes_in_soz}"


In [42]:
def write_eeg_electrodes(ctx: SessionContext):
    nwb = ctx.nwb

    device = nwb.create_device(
        name="NicoletOne EEG System", manufacturer="Natus Medical Incorporated"
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name=f"eeg",
        description=f"EEG electrodes on scalp",
        device=device,
        location="Scalp",
    )

    electrodes = _get_eeg_electrodes(ctx)
    electrodes.apply(lambda row: _add_row_to_eeg_electrodes(nwb, electrode_group, row), axis=1)

In [43]:
def _add_row_to_eeg_electrodes(nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series):
    # got BESA map: +X is anterior, +Y is left, +Z is superior according to <https://eeglab.org/tutorials/ConceptsGuide/coordinateSystem.html>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        location=row["label"],
        reference="Averaged mastoid channels",
        x=-row["x"],
        y=-row["z"],
        z=-row["y"],
        filtering="Passband, 0.5 to 5000 Hz",
    )

In [44]:
def _get_eeg_electrodes(ctx: SessionContext) -> pd.DataFrame:
    labels = _get_eeg_electrode_labels(ctx)
    locations = _get_eeg_electrode_locations(ctx)
    locations_array = np.ndarray(locations.shape)
    locations.read_direct(locations_array)
    df = pd.DataFrame(locations_array, columns=["x", "y", "z"])
    df.insert(0, "label", labels)
    return df.reset_index()

In [45]:
def _get_eeg_electrode_labels(ctx: SessionContext) -> List[str]:
    _assert_all_eeg_electrodes_have_same_labels(ctx)
    session_data = _get_session_data(ctx)
    return session_data.groups["Scalp EEG data"].data_arrays["Scalp_EEG_Data_Trial_01"].dimensions[0].labels

In [46]:
def _assert_all_eeg_electrodes_have_same_labels(ctx: SessionContext):
    data_arrays = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
    electrode_labels = [data_array.dimensions[0].labels for data_array in data_arrays]
    assert len(set(electrode_labels)) == 1

In [47]:
def _get_session_data(ctx: SessionContext) -> nixio.Block:
    return ctx.nix.blocks[f"Data_Subject_{ctx.subject:02}_Session_{ctx.session:02}"]

In [48]:
def _get_eeg_electrode_locations(ctx: SessionContext) -> nixio.DataArray:
    return _get_session_data(ctx).groups["Scalp EEG electrode information"].data_arrays[
        "Scalp_Electrode_EEGLAB_BESA_Coordinates"]

In [49]:
def write_ieeg_electrodes(ctx: SessionContext):
    nwb = ctx.nwb

    device = nwb.create_device(
        name="ATLAS Neurophysiology System", manufacturer="Neuralynx, Inc."
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name="ieeg",
        description=f"iEEG electrodes",
        device=device,
        location="Intracranial",
    )

    electrodes = _get_ieeg_electrodes(ctx)
    electrodes.apply(lambda row: _add_row_to_ieeg_electrodes(nwb, electrode_group, row), axis=1)

In [50]:
def _get_ieeg_electrodes(ctx: SessionContext) -> pd.DataFrame:
    labels = _get_ieeg_electrode_labels(ctx)
    locations = _get_ieeg_electrode_locations(ctx)
    locations_array = np.ndarray(locations.shape)
    locations.read_direct(locations_array)
    df = pd.DataFrame(locations_array, columns=["x", "y", "z"])
    df.insert(0, "label", labels)
    return df.reset_index()

In [51]:
def _get_ieeg_electrode_labels(ctx: SessionContext) -> List[str]:
    return _get_session_data(ctx).groups["iEEG electrode information"].data_arrays[
        "iEEG_Electrode_Manual_Entry"].dimensions[0].labels

In [52]:
def _get_ieeg_electrode_locations(ctx: SessionContext) -> nixio.DataArray:
    return _get_session_data(ctx).groups["iEEG electrode information"].data_arrays[
        "iEEG_Electrode_MNI_Coordinates"]

In [53]:
def _add_row_to_ieeg_electrodes(nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series):
    # Got MNI map: +X is right, +Y is anterior, +Z is superior according to <https://kathleenhupfeld.com/mni-template-coordinate-systems/>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        location=row["label"],
        reference="Common intracranial reference",
        x=-row["y"],
        y=-row["z"],
        z=row["x"],
        filtering="Passband, 0.5 to 1000 Hz",
    )

In [54]:
def write_eeg_measurements(ctx: SessionContext):
    nwb = ctx.nwb
    eeg_electrode_indices = list(range(_get_eeg_electrode_count(ctx)))
    eeg_table_region = nwb.create_electrode_table_region(
        region=eeg_electrode_indices,  # reference row indices 0 to N-1
        description="eeg electrodes",
    )
    trials = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
    for trial in trials:
        metadata = trial.dimensions[1]
        raw_electrical_series = ElectricalSeries(
            name=trial.name,
            data=trial[:].transpose(),
            electrodes=eeg_table_region,
            starting_time=metadata.offset,
            rate=1.0 / metadata.sampling_interval,
        )
        nwb.add_acquisition(raw_electrical_series)

In [55]:
def _get_eeg_electrode_count(ctx: SessionContext) -> int:
    return len(_get_eeg_electrode_labels(ctx))

In [56]:
def write_ieeg_measurements(ctx: SessionContext):
    nwb = ctx.nwb
    min_index = _get_eeg_electrode_count(ctx)
    ieeg_electrode_indices = list(range(min_index, min_index + _get_ieeg_electrode_count(ctx)))
    ieeg_table_region = nwb.create_electrode_table_region(
        region=ieeg_electrode_indices,  # reference row indices 0 to N-1
        description="ieeg electrodes",
    )
    trials = _get_session_data(ctx).groups["iEEG data"].data_arrays
    for trial in trials:
        metadata = trial.dimensions[1]
        raw_electrical_series = ElectricalSeries(
            name=trial.name,
            data=trial[:].transpose(),
            electrodes=ieeg_table_region,
            starting_time=metadata.offset,
            rate=1.0 / metadata.sampling_interval,
        )
        nwb.add_acquisition(raw_electrical_series)


In [57]:
def _get_ieeg_electrode_count(ctx: SessionContext) -> int:
    return len(_get_ieeg_electrode_labels(ctx))

In [58]:
def write_trial_data(ctx: SessionContext):
nwb = ctx.nwb
nwb.add_trial_column(name="trial_number",
                     description="The trial number")
nwb.add_trial_column(name="set_size",
                     description="Number of letters shown during encoding period (4, 6, or 8 letters)")
nwb.add_trial_column(name="probe_letter",
                     description="The letter presented to the participant during the retrieval period")
nwb.add_trial_column(name="correct",
                     description="Whether the participant's response was correct")
nwb.add_trial_column(name="response",
                     description="The participant's answer to the question \"Was the letter at hand present in the encoding set?\"")
nwb.add_trial_column(name="solution",
                     description="The solution to the question \"Was the letter at hand present in the encoding set?\"")
nwb.add_trial_column(name="artifact",
                     description="Whether the trial data was marked as an artifact by the experimenter")

nix = ctx.nix
trials = nix.sections["Session"].sections["Trial properties"].sections
recordings = _get_session_data(ctx).groups["Scalp EEG data"].data_arrays
for trial, recording in zip(trials, recordings):
    trial = trial.props
    metadata = recording.dimensions[1]
    start_time = metadata.offset
    stop_time = start_time + metadata.sampling_interval * recording.shape[1]
    response_time = trial.props["Response time"].values[0]
    # See https://gin.g-node.org/USZ_NCH/Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM/issues/2#issuecomment-3729
    response = str(trial["Response"].values[0])[1] == 1
    solution = int(trial["Match"].values[0]) == 1
    correct = bool(trial["Correct"].values[0])
    nwb.add_trial(
        start_time=start_time,
        stop_time=stop_time,
        trial_number=int(trial["Trial number"].values[0]),
        set_size=int(trial["Set size"].values[0]),
        probe_letter=str(trial["Probe letter"].values[0]),
        response=response,
        response_time=response_time
    solution = solution,
    correct = correct,
    artifact = bool(trial["Artifact"].values[0]),
    )

In [59]:
def write_behavioral_events(ctx: SessionContext):
    nwb = ctx.nwb
    behavior_module = nwb.create_processing_module(
        name="behavior", description="Data for all trials in this session."
    )
    nix = ctx.nix
    trials = nix.sections["Session"].sections["Trial properties"].sections
    for trial in trials:
        time = trial.props["Response time"].values[0]
        events_timestamps = [time]

        time_series = TimeSeries(
            name="response",
            timestamps=events_timestamps,
            description="Time of participant response during retrieval period",
            continuity="instantaneous"
        )

        behavioral_events = BehavioralEvents(time_series=time_series, name="BehavioralEvents")

        behavior_module.add(behavioral_events)

In [60]:
def write_eeg_spike_events(ctx: SessionContext):
    # All events are stimulus imo, except Event_Response, which is the response time we already have
    nwb = ctx.nwb
    pass


In [61]:
def write_nwb(ctx: SessionContext):
    with NWBHDF5IO(f"subject{ctx.subject:02}_session{ctx.session:02}.nwb", "w") as io:
        io.write(ctx.nwb)

In [62]:
if __name__ == "__main__":
    context = main()
    #print(context.nwb.acquisition["Scalp_EEG_Data_Trial_01"])
    #print(context.nwb.electrodes.to_dataframe().to_string())

TypeError: TimeSeries.__init__: missing argument 'data', missing argument 'unit'

DELETE AFTER THIS